In [1]:
from datetime import datetime
from sqlalchemy import (
    MetaData,
    Table,
    Column,
    Integer,
    Numeric,
    String,
    DateTime,
    ForeignKey,
    Boolean,
    create_engine,
    CheckConstraint,
)

metadata = MetaData()

In [2]:
cookies = Table(
    "cookies",
    metadata,
    Column("cookie_id", Integer(), primary_key=True),
    Column("cookie_name", String(50), index=True),
    Column("cookie_recipe_url", String(255)),
    Column("cookie_sku", String(55)),
    Column("quantity", Integer()),
    Column("unit_cost", Numeric(12, 2)),
    CheckConstraint("quantity >=0", name="quantity_positive"),
)
users = Table(
    "users",
    metadata,
    Column("user_id", Integer(), primary_key=True),
    Column("username", String(15), nullable=False, unique=True),
    Column("email_address", String(255), nullable=False),
    Column("phone", String(20), nullable=False),
    Column("password", String(25), nullable=False),
    Column("create_on", DateTime(), default=datetime.now),
    Column(
        "update_on", DateTime(), default=datetime.now, onupdate=datetime.now
    ),
)
orders = Table(
    "orders",
    metadata,
    Column("order_id", Integer()),
    Column("user_id", ForeignKey("users.user_id")),
    Column("shipped", Boolean(), default=False),
)
line_items = Table(
    "line_items",
    metadata,
    Column("line_item_id", Integer(), primary_key=True),
    Column("order_id", ForeignKey("orders.order_id")),
    Column("cookie_id", ForeignKey("cookies.cookie_id")),
    Column("quantity", Integer()),
    Column("extended_cost", Numeric(12, 2)),
)
engine = create_engine("sqlite:///:memory:")
metadata.create_all(engine)
cnn = engine.connect()

In [3]:
from sqlalchemy import select, insert, update

ins = insert(users).values(
    username="cookiemon",
    email_address="mon@cookie.com",
    phone="111-111-1111",
    password="password",
)
result = cnn.execute(ins)
ins = cookies.insert()
lst = [
    {
        "cookie_name": "chocolate chip",
        "cookie_recipe_url": "http://some.awso.me/cookie/recipe.html",
        "cookie_sku": "cc01",
        "quantity": 12,
        "unit_cost": 0.5,
    },
    {
        "cookie_name": "dark chocolate chip",
        "cookie_recipe_url": "http://some.awso.me/cookie/recipe_dark.html",
        "cookie_sku": "cc02",
        "quantity": 1,
        "unit_cost": 0.75,
    },
]
result = cnn.execute(ins, lst)
cnn.commit()

In [4]:
ins = insert(orders).values(user_id=1, order_id=1)
result = cnn.execute(ins)
ins = insert(line_items)
lst = [{"order_id": 1, "cookie_id": 1, "quantity": 9, "extended_cost": 4.5}]
result = cnn.execute(ins, lst)
cnn.commit()

In [5]:
ins = insert(orders).values(user_id=1, order_id=2)
result = cnn.execute(ins)
ins = insert(line_items)
lst = [
    {"order_id": 2, "cookie_id": 1, "quantity": 4, "extended_cost": 1.50},
    {"order_id": 2, "cookie_id": 2, "quantity": 1, "extended_cost": 4.50},
]
result = cnn.execute(ins, lst)
cnn.commit()

In [6]:
def ship_itv1(order_id):
    s = select(line_items.c.cookie_id, line_items.c.quantity)
    s = s.where(line_items.c.order_id == order_id)
    toship = cnn.execute(s)
    for cookie in toship:
        u = update(cookies).where(cookies.c.cookie_id == cookie.cookie_id)
        u = u.values(quantity=cookies.c.quantity - cookie.quantity)
        result = cnn.execute(u)
    u = update(orders).where(orders.c.order_id == order_id)
    u = u.values(shipped=True)
    result = cnn.execute(u)
    cnn.commit()
    print(f"shipped order id: {order_id}")

In [7]:
from sqlalchemy.exc import IntegrityError


def ship_it(order_id):
    s = select(line_items.c.cookie_id, line_items.c.quantity)
    s = s.where(line_items.c.order_id == order_id)
    transaction = cnn.begin()
    toship = cnn.execute(s).fetchall()

    try:
        for cookie in toship:
            u = update(cookies).where(cookies.c.cookie_id == cookie.cookie_id)
            u = u.values(quantity=cookies.c.quantity - cookie.quantity)
            result = cnn.execute(u)
        u = update(orders).where(orders.c.order_id == order_id)
        u = u.values(shipped=True)

        result = cnn.execute(u)
        print(f"Shipped order ID: {order_id}")
        transaction.commit()
    except IntegrityError as error:
        transaction.rollback()
        print(error)

In [8]:
ship_it(1)

Shipped order ID: 1


In [9]:
ship_it(2)

(sqlite3.IntegrityError) CHECK constraint failed: quantity_positive
[SQL: UPDATE cookies SET quantity=(cookies.quantity - ?) WHERE cookies.cookie_id = ?]
[parameters: (4, 1)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [ ]:
ship_it(3)

Shipped order ID: 3
